# HW 2
## Named Entety Recognition and Event Extraction from Literary Fiction

deadline: 12 декабря 2022, 23:59

В этом домашнем задании вы будете работать с корпусом LitBank. Корпус собран из популярных художественных произведений на английском языке и сожержит разметку по именованным сущностям и событиям. Объем корпуса таков: 100 текстов по примерно 2000 слов каждый. 

Корпус описан в статьях:
* David Bamman, Sejal Popat, Sheng Shen, An Annotated Dataset of Literary Entities http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/naacl2019_literary_entities.pdf
* Matthew Sims, Jong Ho Park, David Bamman, Literary Event Detection,  http://people.ischool.berkeley.edu/~dbamman/pubs/pdf/acl2019_literary_events.pdf

Корпус доступен в репозитории проекта:  https://github.com/dbamman/litbank

Статья и код, использованный для извлечения именованных сущностей: 
* Meizhi Ju, Makoto Miwa and Sophia Ananiadou, A Neural Layered Model for Nested Named Entity Recognition, https://github.com/meizhiju/layered-bilstm-crf

Структура корпуса устроена так. 
Первый уровень: 
* entities -- разметка по сущностям
* events -- разметка по сущностям


В корпусе используются 6 типов именованных сущностей: PER, LOC, ORG, FAC, GPE, VEH (имена, локации, организации, помещения, топонимы, средства перемещния), допускаются вложенные сущности. 

События выражается одним словом - *триггером*, которое может быть глагом, прилагательным и существительным. В корпусе описаны события, которые действительно происходят и не имеют гипотетического характера. 
Пример: she *walked* rapidly and resolutely, здесь *walked* -- триггер события. Типы событий не заданы. 



Второй уровень:
* brat -- рабочие файлы инструмента разметки brat, ann-файлы содержат разметку, txt-файлы – сырые тексты 
* tsv -- tsv-файлы содержат разметку в IOB формате,


В статье и репозитории вы найдете идеи, которые помогут вам выполнить домашнее задание. Их стоит воспринимать как руководство к действию, и не стоит их копировать и переиспользовать. Обученные модели использовать не нужно, код для их обучения можно использовать как подсказку. 

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 4-х человек.
2. Домашнее задание сдается через github.classroom, инвайты будут высланы.
3. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо ipython-тетрадке. 
4. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
5. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
6. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
7. Плагиат и любое недобросоветсное цитирование приводит к обнуление оценки. 


## Base imports

In [1]:
from typing import *
import re
from pathlib import Path
import shutil
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import joblib
%load_ext autoreload
%autoreload 2

## Dataset download and init

In [2]:
data_dir = Path('data/litbank')

if not data_dir.exists():
    %cd data
    !git clone https://github.com/dbamman/litbank
    %cd ..

In [3]:
from utils import LitbankDataset

dataset = LitbankDataset(data_dir)

# Словари с датафреймом разметки для каждого текста
entity_df_dict = dataset.df_dicts['entities']
event_df_dict = dataset.df_dicts['events']
# Объединенные датафреймы
entity_df_all = dataset.dfs['entities']
event_df_all = dataset.dfs['events']

In [80]:
entity_df_dict.keys()

dict_keys(['541_the_age_of_innocence_brat.tsv', '2891_howards_end_brat.tsv', '2775_the_good_soldier_brat.tsv', '6593_history_of_tom_jones_a_foundling_brat.tsv', '44_the_song_of_the_lark_brat.tsv', '32_herland_brat.tsv', '271_black_beauty_brat.tsv', '4300_ulysses_brat.tsv', '351_of_human_bondage_brat.tsv', '940_the_last_of_the_mohicans_a_narrative_of_1757_brat.tsv', '238_dear_enemy_brat.tsv', '4217_a_portrait_of_the_artist_as_a_young_man_brat.tsv', '2084_the_way_of_all_flesh_brat.tsv', '233_sister_carrie_a_novel_brat.tsv', '974_the_secret_agent_a_simple_tale_brat.tsv', '1695_the_man_who_was_thursday_a_nightmare_brat.tsv', '5348_ragged_dick_or_street_life_in_new_york_with_the_bootblacks_brat.tsv', '514_little_women_brat.tsv', '730_oliver_twist_brat.tsv', '78_tarzan_of_the_apes_brat.tsv', '160_the_awakening_and_selected_short_stories_brat.tsv', '345_dracula_brat.tsv', '45_anne_of_green_gables_brat.tsv', '24_o_pioneers_brat.tsv', '2641_a_room_with_a_view_brat.tsv', '36_the_war_of_the_world

In [74]:
entity_df_all

,token,label0,label1,label2,label3,label4
0,Book,O,O,O,O,0
1,I,O,O,O,O,0
2,I.,O,O,O,O,0
3,On,O,O,O,O,0
4,a,O,O,O,O,0
...,...,...,...,...,...,...
2058,shocking,O,O,O,O,0
2059,walk,O,O,O,O,0
2060,.,O,O,O,O,0
2061,”,O,O,O,O,0



## Часть 1. [3 балла] Эксплоративный анализ
1. Найдите топ 10 (по частоте) именованных сущностей каждого из 6 типов.
2. Найдите топ 10 (по частоте) частотных триггеров событий.
3. Кластеризуйте все уникальные триггеры событий, используя эмбеддинги слов и любой алгоритм кластеризации (например, агломеративный иерархический алгоритм кластеризации) и попробуйте проинтерпретировать кластеры: есть ли очевидные типы событий?

[бонус] Визуализируйте полученные кластеры с помощью TSNE или UMAP

[бонус] Постройте тематическую модель по корпусу и сравните кластеры тригеров и выделенные темы: есть ли схожие паттерны в тематической модели и в стурктуре кластеров?

В следующих частях домашнего задания вам понадобится train-test-dev разбиение. Авторы статей предлагают следующую структуру разбиения: обучающее множество – 80 книг, валидационное – 10 книг, тестовое – 10 книг. Предложения из одного источника не должны попадать в разные сегменты разбиения.


1. Найдите топ 10 (по частоте) именованных сущностей каждого из 6 типов.

Пройдем по каждой строке датафрейма с разметкой и будем сохранять, сколько раз каждое слово являлось той или иной сущностью

In [94]:
from collections import defaultdict, Counter

entity_words_dict = defaultdict(Counter)

for i, (token, *labels) in entity_df_all.iterrows():
    for label in labels:
        if label in [0, '0', 'O']:
            continue
        # Игнорируем X в X-NER
        label = label.split('-')[-1]
        entity_words_dict[label][token] += 1

print('Топ-10 по встречаемости cущностей\n')
for label, counter in entity_words_dict.items():
    print(label)
    print(counter.most_common(10))
    print()

Топ-10 по встречаемости cущностей

PER
[('the', 2148), ('a', 1137), ('of', 776), (',', 670), ('his', 406), ('man', 361), ('who', 309), ('Mr.', 307), ('and', 305), ('her', 289)]

FAC
[('the', 1194), ('a', 260), ('of', 223), ('house', 153), (',', 138), ('room', 100), ('home', 83), ('The', 80), ('and', 76), ('in', 74)]

GPE
[('the', 172), ('town', 64), ('of', 55), ('London', 40), ('a', 39), ('England', 38), ('country', 34), ('New', 27), ('village', 27), (',', 26)]

VEH
[('the', 97), ('a', 33), ('ship', 21), ('car', 15), ('train', 14), ('ships', 12), ('The', 11), ('carriage', 10), ('_', 10), ('boats', 8)]

LOC
[('the', 828), ('of', 195), ('a', 122), ('world', 112), (',', 64), ('sea', 56), ('and', 53), ('river', 50), ('country', 47), ('this', 42)]

ORG
[('the', 86), ('of', 28), ('a', 21), ('army', 20), (',', 19), ('and', 10), ('that', 7), ('his', 7), ('Church', 7), ('an', 6)]



2. Найдите топ 10 (по частоте) частотных триггеров событий.

То же самое, но еще проще

In [4]:
event_words_counter = Counter()

for i, (token, label) in event_df_all.iterrows():
    if label == 'EVENT':
        event_words_counter[token] += 1

print('Топ-10 по встречаемости событий\n')

print(event_words_counter.most_common(10))

Топ-10 по встречаемости событий

[('said', 464), ('came', 95), ('looked', 92), ('went', 92), ('asked', 69), ('heard', 63), ('saw', 59), ('cried', 59), ('took', 56), ('turned', 55)]


3. Кластеризуйте все уникальные триггеры событий, используя эмбеддинги слов и любой алгоритм кластеризации (например, агломеративный иерархический алгоритм кластеризации) и попробуйте проинтерпретировать кластеры: есть ли очевидные типы событий?


In [26]:
from sklearn.cluster import AgglomerativeClustering
import gensim.downloader


word2vec_model = gensim.downloader.load('glove-wiki-gigaword-50')
words = list(event_words_counter.keys())
X = np.array([word2vec_model.get_vector(word) for word in words
              if word in word2vec_model.key_to_index])

In [46]:
cluster_model = AgglomerativeClustering(distance_threshold=30, n_clusters=None)
cluster_model.fit(X)

AgglomerativeClustering(distance_threshold=30, n_clusters=None)

In [48]:
for cluster in range(cluster_model.n_clusters_):

    non_cluster_words = [words[i] for i, label in enumerate(cluster_model.labels_)
                         if label != cluster]
    cluster_words_counter = event_words_counter.copy()
    for word in non_cluster_words:
        cluster_words_counter.pop(word)

    print(f'Топ слов по встречаемости из кластера {cluster}')
    most_common = cluster_words_counter.most_common(10)
    print(most_common)

Топ слов по встречаемости из кластера 0
[('came', 95), ('looked', 92), ('went', 92), ('asked', 69), ('saw', 59), ('turned', 55), ('come', 37), ('returned', 34), ('see', 34), ('replied', 30)]
Топ слов по встречаемости из кластера 1
[('told', 51), ('fire', 34), ('met', 26), ('reached', 26), ('felt', 25), ('spoke', 25), ('called', 24), ('stopped', 22), ('saying', 18), ('fell', 18)]
Топ слов по встречаемости из кластера 2
[('took', 56), ('answered', 45), ('put', 44), ('walked', 30), ('brought', 27), ('seen', 21), ('remember', 20), ('died', 19), ('sent', 18), ('death', 17)]
Топ слов по встречаемости из кластера 3
[('found', 49), ('thought', 38), ('looking', 30), ('glance', 23), ('observed', 22), ('pulled', 17), ('says', 16), ('watched', 13), ('explained', 12), ('promised', 11)]
Топ слов по встречаемости из кластера 4
[('said', 464), ('heard', 63), ('got', 34), ('made', 31), ('tried', 25), ('shook', 18), ('set', 17), ('walking', 14), ('struck', 13), ('fog', 13)]
Топ слов по встречаемости из 

0-ой кластер: глаголы движения

1-ый кластер: много слов связанных с разговорами

2-ый кластер: много слов связанных с манипуляцией предметами (положить/взять)

3-ый кластер: слова связанные с наблюдением

5-ый кластер: слова связанные с грустью

6-ый кластер: слова связанные с дикой живностью (охота, выращивать, поймать)


## Часть 2. [5 баллов] Извлечение именованных сущностей
1. Используйте стандартную (любую предобученную) модель для извлечения именованных сущностей. Продемонстрируйте, какие сущности она извлекает. Вычислите качество работы модели на токенах и на спанах сущностей. Для вычисления качества работы модели используйте seqeval (умеет работать с Huggingface). Какая из метрик получилась выше?

2. Дообучите BERT для извлечения именованных сущностей.

2. (Как вариант альтернативный обучению BERT) Можно обучить модель CNN-BiLSTM-CRF, для извлечения именованных *низкоуровневых именованных сущностей*, т.е. для самых коротких из вложенных сущностей.
Модель устроена так: сверточная сеть на символах + эмбеддинги слов + двунаправленная LSTM сеть (модель последовательности) + CRF (глобальная нормализация)

[бонус] Используйте модель для извлечения вложенных именованных сущностей [Ju et al., 2018] Можно использовать модель из статьи, можно также вместо эмбеддингов слов использовать ELMo и/или BERT.


## Часть 3. [2 балла] Извлечение событий

1. Используйте BiLSTM на эмбеддингах слов для извлечения триггеров событий.

2. Замените часть модели на  словах  на ELMo и/или BERT.  Должна получиться модель ELMo / BERT + BiLSTM.

[бонус] Предобучите BiLSTM как языковую модель. Дообучите ее для извлечения триггеров.

[бонус] Дообучите BERT для извлечения триггеров событий.


## Часть 4. [2 балла] Одновременное извлечение именованных сущностей и событий
1. Обучите модель для совместного извлечения именованных сущностей и триггеров событий. У модели должен быть общий энкодер (например, BERT, CNN + BiLSMT, ELMo + BiLSTM, BERT + BiLSTM) и два декодера: один отвечает за извлечение именнованных сущностей, другой отвечает за извлечение триггеров событий.

[бонус] Добавьте в модель механизм внимания, так, как это покажется вам разумным.

[бонус] Визуализируйте карты механизма внимания.


## Часть 5. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?